# 02 Q-agent

[Q-öğrenme (Q-learning)](https://www.gatsby.ucl.ac.uk/~dayan/papers/cjch.pdf)  ([Chris Walkins](www.cs.rhul.ac.uk/~chrisw/) ve  [Peter Dayan](https://en.wikipedia.org/wiki/Peter_Dayan) tarafından) optimal q-değeri fonksiyonunu bulan bir algoritmadır.

Kısım 1'de söylediğimiz gibi, bir π politikasıyla ilişkili **q-değer fonksiyonu Q(s, a), ajanın s durumunda a eylemini yaptığı ve ardından π politikasını izlediği zaman ajanın almayı beklediği toplam ödüldür.**

Optimum q-değer fonksiyonu **Q*(s, a)**, optimal politika π* ile ilişkili q-değer fonksiyonudur.

**Q*(s, a)**'yı biliyorsanız, π*: sonucunu çıkarabilirsiniz, yani bir sonraki eylem olarak mevcut s durumu için Q*(s, a)'yı maksimize eden eylemi seçersiniz.

Q-öğrenme, keyfi bir başlangıç **Q⁰(s, a)** tahmininden başlayarak, optimal q-değer fonksiyonu **Q*(s, a)** için daha iyi ve daha iyi yaklaşımları hesaplamak için yinelemeli bir algoritmadır.

<img src="https://miro.medium.com/max/1260/0*323DNjYPk23v3Mpq.png" width=500px/>

Sonlu sayıda durum ve eylem içeren `Taxi-v3` gibi bir tablo ortamında, bir **q işlevi esasen bir matristir**. **Durumlar kadar satır ve eylemler kadar sütun içerir, yani 500 x 6**.

Tamam, ama Q⁰(s, a)'dan sonraki Q¹(s, a) yaklaşımını tam olarak nasıl hesaplarsınız?

Bu, Q-öğrenmedeki anahtar formüldür:

<img src="https://miro.medium.com/max/1260/1*ODDuOVNWybX6g0CI4i2Uiw.png" width=500px/>


q-agent çevrede gezinirken ve bir sonraki s' durumunu ve r ödülünü gözlemledikçe, q-değer matrisinizi bu formülle güncellersiniz.

## *Bu formüldeki öğrenme oranı 𝛼 nedir?*

**Öğrenme oranı (learning rate)** (makine öğreniminde her zamanki gibi), q-fonksiyonu güncellemelerin ne kadar büyük olduğunu kontrol eden küçük bir sayıdır. Ayarlamanız gerekir, çünkü çok büyük bir değer dengesiz eğitime neden olur ve çok küçük yerel minimumlardan kaçmak için yeterli olmayabilir.

## *Ve bu indirim faktörü 𝛾?*

**İndirim faktörü (discount factor)**, ajanımızın yakın gelecekteki ödüllere göre uzak gelecekteki ödülleri ne kadar önemsediğini belirleyen 0 ile 1 arasında bir (hiper) parametredir.

- 𝛾=0 olduğunda, temsilci yalnızca anında ödülü maksimize etmekle ilgilenir. Hayatta olduğu gibi, anında ödülü en üst düzeye çıkarmak, optimal uzun vadeli sonuçlar için en iyi reçete değildir. Bu, RL aracılarında da olur.

- 𝛾=1 olduğunda, temsilci her eylemini gelecekteki tüm ödüllerinin toplamına göre değerlendirir. Bu durumda temsilci, anlık ödülleri ve gelecekteki ödülleri eşit olarak değerlendirir.

İndirim faktörü tipik olarak bir ara değerdir, örn. 0.6.

## Özetle, eğer

- yeterince uzun eğitim
- iyi bir öğrenme oranı ve indirim faktörü ile
- ve ajan durum uzayını yeterince araştırıyor
- ve q-değer matrisini Q-öğrenme formülüyle güncellersiniz

ilk yaklaşımınız sonunda optimal q-matrise yakınsar. işte!

O zaman bir Q-agent için bir Python sınıfı uygulayalım.


👉Akıllı bir taksi şoförü yetiştirmek için Q-learning'i kullanalım.

👉Akıllı bir taksi şoförü ceza almamalı (yani kaza yapmamalı) ve sürüşü tamamlamak için gereken zaman adımlarını en aza indirmelidir.

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
%config InlineBackend.figure_format = 'svg'

## Environment 🌎

In [ ]:
import gym
env = gym.make("Taxi-v3").env

## Q-agent 🤖🧠

In [ ]:
import numpy as np

class QAgent:

    def __init__(self, env, alpha, gamma):
        self.env = env

        # table with q-values: n_states * n_actions
        self.q_table = np.zeros([env.observation_space.n,
                                 env.action_space.n])

        # hyper-parameters
        self.alpha = alpha  # learning rate
        self.gamma = gamma  # discount factor

    def get_action(self, state):
        """"""
        return np.argmax(self.q_table[state])

    def update_parameters(self, state, action, reward, next_state):
        """"""
        # Q-learning formula
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state])
        new_value = \
            old_value + \
            self.alpha * (reward + self.gamma * next_max - old_value)

        # update the q_table
        self.q_table[state, action] = new_value

In [ ]:
# hyper-parameters (hiper parametreler)

# RL problems are full of these hyper-parameters. 
# (RL problemleri bu hiper parametrelerle doludur.)

# For the moment, trust me when I set these values.
# We will later play with these and see how they impact learning.

alpha = 0.1
gamma = 0.6

agent = QAgent(env, alpha, gamma)

## Training loop 🎡

In [ ]:
import random
from tqdm import tqdm

# exploration vs exploitation prob
epsilon = 0.1

n_episodes = 10000

# For plotting metrics
timesteps_per_episode = []
penalties_per_episode = []


for i in tqdm(range(0, n_episodes)):
    
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        
        if random.uniform(0, 1) < epsilon:
            # Explore action space
            action = env.action_space.sample()
        else:
            # Exploit learned values
            action = agent.get_action(state)
        
        next_state, reward, done, info = env.step(action) 
        
        agent.update_parameters(state, action, reward, next_state)
        
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    timesteps_per_episode.append(epochs)
    penalties_per_episode.append(penalties)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (12, 4))
ax.set_title("Timesteps to complete ride")    
pd.Series(timesteps_per_episode).plot(kind='line')
plt.show()

fig, ax = plt.subplots(figsize = (12, 4))
ax.set_title("Penalties per ride")    
pd.Series(penalties_per_episode).plot(kind='line')
plt.show()

## Sabit bir `state = 123`'ten başlayarak bu sürücüyü değerlendirelim (evaluate)

In [ ]:
# set initial state of the environment
state = 123
env.s = state

epochs = 0
penalties = 0
reward = 0

# store frames to latter plot them
frames = []

done = False

while not done:
    
    action = agent.get_action(state)
    
    next_state, reward, done, info = env.step(action)
    agent.update_parameters(state, action, reward, next_state)

    if reward == -10:
        penalties += 1
    
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    state = next_state

    epochs += 1
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1} of {len(frames)}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

## Bu eğitimli ajanı 100 bölüm üzerinden değerlendirelim

In [ ]:
import random
from tqdm import tqdm

# exploration vs exploitation prob
epsilon = 0.05

n_episodes = 100

# For plotting metrics
timesteps_per_episode = []
penalties_per_episode = []


for i in tqdm(range(0, n_episodes)):
    
    state = env.reset()      
    
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        
        if random.uniform(0, 1) < epsilon:
            # Explore action space
            action = env.action_space.sample()
        else:
            # Exploit learned values
            action = agent.get_action(state)
        
        next_state, reward, done, info = env.step(action)
        
        agent.update_parameters(state, action, reward, next_state)
                      
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
            
    timesteps_per_episode.append(epochs)
    penalties_per_episode.append(penalties)

In [ ]:
print(f'Avg steps to complete ride: {np.array(timesteps_per_episode).mean()}')
print(f'Avg penalties to complete ride: {np.array(penalties_per_episode).mean()}')